In [5]:
#os and i/o
import os
import numpy as np
import glob
from os.path import abspath
import csv

#scientific computing
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats, optimize
from pandas import DataFrame, Series
import seaborn as sns
import random as rd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import scipy.stats

#ipython add-ons
from IPython.parallel import Client
from IPython.display import Image
import multiprocessing

##nipype
import nibabel as nib
from nipype.pipeline.engine import Node, MapNode, Workflow
from nipype.interfaces.io import DataGrabber, DataFinder, DataSink
from nipype.interfaces import fsl
from nipype.interfaces.fsl import MELODIC
%matplotlib inline

/home/iballard/anaconda/lib/python2.7/site-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)


In [6]:
rc = Client()

In [7]:
#preliminary housekeeping
home_dir = '/data/home/iballard/fd/'
subj_file = home_dir + 'subjects.txt'
sub_list = list(np.loadtxt(subj_file,'string'))
os.chdir(home_dir)

In [4]:
def apply_melodic(in_tuple):
    sub, exp, run = in_tuple
    
    sub_path = home_dir + 'analysis/' + exp + '/' + sub + '/preproc/run_' + run + '/'
    
    if os.path.exists(sub_path):
        scan = sub_path + 'unsmoothed_timeseries.nii.gz'
        mask = sub_path + 'functional_mask.nii.gz'
        out_dir = sub_path + 'melodic'
        
        if not os.path.exists(out_dir):
            os.mkdir(out_dir)
        
        if os.listdir(out_dir) == []: #directory empty
            melodic_setup = MELODIC(in_files = scan, no_bet = True, bg_threshold = 10, tr_sec = 1.5,
                                   mm_thresh = .5, out_stats = True, report = False, mask = mask,
                                   out_dir = out_dir)
            melodic_setup.run()

In [16]:
in_tuples = []
for sub in sub_list:
    if sub in ['fd_104']:
        for exp in ['loc']:
            for run in range(1,2):
                in_tuples.append((sub,exp,str(run)))
print in_tuples

[('fd_104', 'loc', '1')]


In [17]:
dview = rc[0:18]
dview.block = True

dview.push(dict(home_dir=home_dir,
                ))
dview.execute("import numpy as np")
with dview.sync_imports():
    import os
    import numpy
    from nipype.interfaces.fsl import MELODIC
    
#fit lots of models
dview.map_sync(apply_melodic,in_tuples)


importing os on engine(s)
importing numpy on engine(s)
importing MELODIC from nipype.interfaces.fsl on engine(s)


[None]

In [18]:
exp = 'loc'
for sub in sub_list:
    for run in range(1,3):
        sub_path = home_dir + 'analysis/' + exp + '/' + sub + '/preproc/run_' + str(run) + '/melodic/'
        out_f = sub_path + 'melodic_IC.nii.gz'
        if not os.path.exists(out_f):
            print sub,run